# Imports

In [78]:
using MLDatasets: MNIST
using Knet, IterTools, MLDatasets
using Dictionaries
using TimerOutputs
using TimerOutputs
using JSON
using Printf
using Knet:minibatch
using Knet:minimize
using Knet
using Knet: Param
using Knet: Knet, dir, accuracy, progress, sgd, gc, Data, nll, relu
using Flatten
using Flux.Data;
using Flux, Statistics

# Processing Data

In [79]:
# This loads the MNIST handwritten digit recognition dataset. This code is based off the Knet Tutorial Notebook. 
xtrn,ytrn = MNIST.traindata(Float32)
xtst,ytst = MNIST.testdata(Float32)
println.(summary.((xtrn,ytrn,xtst,ytst)));

28×28×60000 Array{Float32, 3}
60000-element Vector{Int64}
28×28×10000 Array{Float32, 3}
10000-element Vector{Int64}


In [80]:
xtrn = reshape(xtrn, 784, 60000 ) 
xtst = reshape(xtst, 784, 10000 )
println(summary.((xtrn, xtst))) # can see the data that is flattened 

("784×60000 Matrix{Float32}", "784×10000 Matrix{Float32}")


In [81]:
#Preprocessing targets: one hot vectors
# ytrn = onehotbatch(ytrn, 0:9)
# ytst = onehotbatch(ytst, 0:9)

# Batch Processing

In [82]:
train_loader = DataLoader((xtrn, ytrn), batchsize=128);
test_loader = DataLoader((xtst, ytst), batchsize = 128)

DataLoader{Tuple{Matrix{Float32}, Vector{Int64}}, Random._GLOBAL_RNG, Val{nothing}}((Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [7, 2, 1, 0, 4, 1, 4, 9, 5, 9  …  7, 8, 9, 0, 1, 2, 3, 4, 5, 6]), 128, false, true, false, false, Val{nothing}(), Random._GLOBAL_RNG())

In [83]:
length(test_loader)

79

In [84]:
(x,y) = first(train_loader) #gives the first minibatch from training dataset
println.(summary.((x,y)));

784×128 Matrix{Float32}
128-element Vector{Int64}


# Define Dense Layer

In [85]:
struct Dense1; w; b; f; end
Dense1(i,o; f=relu) = Dense1(param(o,i), param0(o), f)
(d::Dense1)(x) = d.f.(d.w * mat(x) .+ d.b)

# Define Chain Layer


In [86]:
# Define a chain of layers and a loss function:
struct Chain; layers; end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = nll(c(x),y)

# Define the Model

In [87]:
model = Chain((Dense1(784, 100), Dense1(100, 10), identity))

Chain((Dense1(P(Matrix{Float32}(100,784)), P(Vector{Float32}(100)), Knet.Ops20.relu), Dense1(P(Matrix{Float32}(10,100)), P(Vector{Float32}(10)), Knet.Ops20.relu), identity))

# Training and Evaluating

In [88]:
model(x) #checking if training is working

10×128 Matrix{Float32}:
 0.146128  0.251807   0.0944941  0.0        …  0.0       0.0       0.166544
 0.908352  0.975714   0.392101   1.07251       1.10933   0.714653  1.1014
 0.0       0.0        0.0        0.0           0.0       0.37319   0.394873
 0.0       0.0403631  0.227371   0.218725      0.187911  0.0       0.0
 0.0       0.0        0.0943293  0.0           0.0       0.0       0.0
 0.0       0.0        0.0        0.415859   …  0.0       0.0       0.0
 0.0       0.0        0.0        0.0           0.0       0.459048  0.0691352
 0.189706  0.802615   0.69828    0.0941242     0.455495  0.444184  0.604847
 0.834217  0.47262    0.0        0.273323      0.575527  0.126404  0.402153
 0.0       0.0        0.0        0.0           0.0       0.0       0.0

In [89]:

loss(xtst, ytst) = nll(model(xtst), ytst)
evalcb = () -> (loss(xtst, ytst)) #function that will be called to get the loss 
const to = TimerOutput() # creating a TimerOutput, keeps track of everything


@timeit to "Train Total" begin
    for epoch in 1:10
        train_epoch = epoch > 1 ? "train_epoch" : "train_ji"
        @timeit to train_epoch begin
            progress!(adam(model, train_loader; lr = 1e-3))
        end
        
        evaluation = epoch > 1 ? "evaluation" : "eval_jit"
        @timeit to evaluation begin
            accuracy(model, test_loader)
        end 
        
    end 
end 



┣████████████████████┫ [100.00%, 469/469, 00:01/00:01, 350.61i/s] 
┣████████████████████┫ [100.00%, 469/469, 00:01/00:01, 381.14i/s] 
┣████████████████████┫ [100.00%, 469/469, 00:01/00:01, 383.71i/s] 
┣████████████████████┫ [100.00%, 469/469, 00:01/00:01, 393.42i/s] 
┣████████████████████┫ [100.00%, 469/469, 00:01/00:01, 362.42i/s] 
┣████████████████████┫ [100.00%, 469/469, 00:01/00:01, 352.92i/s] 
┣████████████████████┫ [100.00%, 469/469, 00:01/00:01, 322.49i/s] 
┣████████████████████┫ [100.00%, 469/469, 00:01/00:01, 355.11i/s] 
┣████████████████████┫ [100.00%, 469/469, 00:01/00:01, 357.56i/s] 
┣████████████████████┫ [100.00%, 469/469, 00:01/00:01, 375.06i/s] 


In [90]:

final_train_loss = evalcb()
final_eval_accuracy = accuracy(model, test_loader)

# see the overall loss
println("Overall Loss: ", final_train_loss) 
println("Overall Accuracy: ", final_eval_accuracy ) #see the overall accuracy


Overall Loss: 0.09791139
Overall Accuracy: 0.9695121951219512


# Getting Metrics

In [96]:


show(to, allocations = true, compact = true) #see the time it took for training and evaluating the model

 ─────────────────────────────────────────────────────────
                               Time          Allocations  
                         ───────────────   ───────────────
     Total measured:          22.9s            10.6GiB    

 Section         ncalls     time    %tot     alloc    %tot
 ─────────────────────────────────────────────────────────
 Train Total          1    13.5s  100.0%   10.5GiB  100.0%
   train_epoch        9    11.6s   85.8%   9.15GiB   86.7%
   train_ji           1    1.35s   10.0%   1.02GiB    9.7%
   evaluation         9    505ms    3.7%    352MiB    3.3%
   eval_jit           1   60.6ms    0.4%   39.1MiB    0.4%
 ─────────────────────────────────────────────────────────

In [97]:
#average epoch training time converted to seconds from nanoseconds
average_train_epoch_time = (TimerOutputs.time(to["Train Total"]["train_epoch"]))/(1e+9 *9)
total_train_time = TimerOutputs.time(to["Train Total"])/(1e+9)
average_batch_inference_time = TimerOutputs.time(to["Train Total"]["evaluation"])/(length(test_loader)*1e+6*9)

average_train_epoch_time


1.2902278333333332

In [98]:

#getting dictionary to format the metrics
metrics = Dict("model_name" => "MLP",
 "framework_name"=>"Knet",
  "dataset" => "MNIST Digits", 
    "task" => "classifcation",
    "average_epoch_training_time" => average_train_epoch_time,
    "total_training_time" => total_train_time,
    "average_batch_inference_time" => average_batch_inference_time,
    "final_training_loss" => final_train_loss,
    "final_evaluation_accuracy" => final_eval_accuracy
)

Dict{String, Any} with 9 entries:
  "task"                         => "classifcation"
  "framework_name"               => "Knet"
  "final_evaluation_accuracy"    => 0.969512
  "average_epoch_training_time"  => 1.29023
  "total_training_time"          => 13.5294
  "final_training_loss"          => 0.0979114
  "model_name"                   => "MLP"
  "dataset"                      => "MNIST Digits"
  "average_batch_inference_time" => 0.709791

In [99]:

stringdata = JSON.json(metrics)

#will allow the metrics to be entered into a file 

open("M1-Knet-mlp.json", "w") do f
    write(f, stringdata)
    end 

dict2 = Dict()
open("M1-Knet-mlp.json", "r") do f
    global dict2
    dict2 = JSON.parse(f)
end

Dict{String, Any} with 9 entries:
  "task"                         => "classifcation"
  "framework_name"               => "Knet"
  "final_evaluation_accuracy"    => 0.969512
  "average_epoch_training_time"  => 1.29023
  "total_training_time"          => 13.5294
  "final_training_loss"          => 0.0979114
  "model_name"                   => "MLP"
  "dataset"                      => "MNIST Digits"
  "average_batch_inference_time" => 0.709791

In [100]:
pwd() #checking directory 

"C:\\Users\\Yash"